In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import os
from google.colab import files

projectID = 'finalproject-ecmms'
location = 'US'
client = bigquery.Client(project=projectID, location=location)

auth.authenticate_user()

print('authenticated')

In [ ]:
from google.cloud import bigquery
from google.colab import files

client = bigquery.Client()

queries = {
    '0-andamento_profitti': """
        SELECT
          FORMAT_TIMESTAMP("%Y-%m", created_at) AS year_month,
          ROUND(SUM(oi.sale_price - p.cost)) AS profit
        FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
          ON oi.product_id = p.id
        WHERE oi.status = 'Complete'
        GROUP BY year_month
        ORDER BY year_month
    """,
    '0-conteggio_ordini': """
        SELECT
          EXTRACT(DATE FROM created_at) AS day,
          COUNT(*) AS total_orders
        FROM `bigquery-public-data.thelook_ecommerce.orders`
        WHERE status = 'Complete'
        GROUP BY day
        ORDER BY total_orders DESC
    """,
    '0-media_tempo_spedizione': """
        SELECT u.country AS country,
          EXTRACT(YEAR FROM o.created_at) AS year,
          AVG(DATE_DIFF(o.shipped_at, o.created_at, DAY)) AS avg_days
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        GROUP BY country, year
        ORDER BY year
    """
}

for query_name, query in queries.items():
    query_job = client.query(query)
    results = query_job.result().to_dataframe()
    filename = f'{query_name}.csv'
    results.to_csv(filename, index=False)
    print(f'Risultati "{query_name}" salvati come csv')
    files.download(filename)

print('Download completato')

